### faster whisper

In [ ]:
from faster_whisper import WhisperModel

model_size = "tiny"
model_size_2 = "medium"

# Run on GPU with FP16
# model = WhisperModel(model_size, device="cuda", compute_type="float16")
# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
model = WhisperModel(model_size, device="cpu", compute_type="int8")
model_2 = WhisperModel(model_size_2, device="cpu", compute_type="int8")


/home/b0990106x/miniconda3/envs/dpo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import string
from jiwer import wer

def reward_wer(reference, hypothesis):
    raw_wer = wer(reference, hypothesis)
    normalized = min(raw_wer, 1.0)
    return 1 - normalized

def test(asr_model):
    segments, info = asr_model.transcribe("output/1118-2248/example_generate_data_4_item_1.wav", beam_size=1)
    # print("Detected language '%s' with probability %f" % (info.language, info.language_probability)) 
    segments = list(segments)    
    modified_text = ''.join(char for char in segments[0].text if char not in string.punctuation).lower().strip()
    # ground_truth = "neighboring fields"
    ground_truth = "goodbye"
    reward = reward_wer(modified_text, ground_truth)
    print(f"Reward: {reward:.2%}, Text: {modified_text}")
    return reward
    
 # count time
import time
start = time.time()
rewards = [test(model) for _ in range(10)]
print(f"Average reward: {sum(rewards) / len(rewards):.2%}")
print(f"Time: {time.time() - start:.2f}s")

start = time.time()
rewards = [test(model_2) for _ in range(10)]
print(f"Average reward: {sum(rewards) / len(rewards):.2%}")
print(f"Time: {time.time() - start:.2f}s")


Reward: 0.00%, Text: bye
Reward: 0.00%, Text: bye
Reward: 0.00%, Text: bye
Reward: 0.00%, Text: bye
Reward: 0.00%, Text: bye
Reward: 0.00%, Text: bye
Reward: 0.00%, Text: bye
Reward: 0.00%, Text: bye
Reward: 0.00%, Text: bye
Reward: 0.00%, Text: bye
Average reward: 0.00%
Time: 4.46s
Reward: 0.00%, Text: bye bye
Reward: 0.00%, Text: bye bye
Reward: 0.00%, Text: bye bye
Reward: 0.00%, Text: bye bye
Reward: 0.00%, Text: bye bye
Reward: 0.00%, Text: bye bye
Reward: 0.00%, Text: bye bye
Reward: 0.00%, Text: bye bye
Reward: 0.00%, Text: bye bye
Reward: 0.00%, Text: bye bye
Average reward: 0.00%
Time: 39.64s


In [11]:
model_size_3 = "medium"
model_3 = WhisperModel(model_size_3, device="cuda", compute_type="float16")
start = time.time()
rewards = [test(model_3) for _ in range(10)]
print(f"Average reward: {sum(rewards) / len(rewards):.2%}")
print(f"Time: {time.time() - start:.2f}s")

Reward: 100.00%, Text: goodbye
Reward: 100.00%, Text: goodbye
Reward: 100.00%, Text: goodbye
Reward: 100.00%, Text: goodbye
Reward: 100.00%, Text: goodbye
Reward: 100.00%, Text: goodbye
Reward: 100.00%, Text: goodbye
Reward: 100.00%, Text: goodbye
Reward: 100.00%, Text: goodbye
Reward: 100.00%, Text: goodbye
Average reward: 100.00%
Time: 2.60s


In [3]:
from jiwer import wer

text1 = " Neighboring fields"
text2 = "neighboring fields"
error_rate = wer(text1, text2)
print(error_rate)

print(f"Word Error Rate: {error_rate:.2%}")


0.5
Word Error Rate: 50.00%


In [4]:
def reward_wer(reference, hypothesis):
    raw_wer = wer(reference, hypothesis)
    normalized = min(raw_wer, 1.0)
    return 1 - normalized

text1 = "make a ring for you"
text2 = "neighboring fields"

reward = reward_wer(text1, text2)
print(f"Reward: {reward:.2f}")

Reward: 0.00


In [5]:
import string
text = "Neighboring fields." # 8
text = "But I will be, in a minute." # 16
text = "Will you?" # 65
text = "Do they?" # 100
text = "Don't you?" # 102

text = "I don't know" # 105
text = "Why is it?" # 112
text = "Why not?" # 132
text = "Good-bye" # 140
text = "The idea!" # 184


modified_text = ''.join(char for char in text if char not in string.punctuation).lower().strip()
print(modified_text)

the idea


In [6]:
import string

text_list = ["Neighboring fields.", "But I will be, in a minute.", "Will you?", "Do they?", "Don't you?", "I don't know", "Why is it?", "Why not?", "Good-bye", "The idea!"]
index_list = [8, 16, 65, 100, 102, 105, 112, 132, 140, 184]
for i, text in enumerate(text_list):
    modified_text = ''.join(char for char in text if char not in string.punctuation).lower().strip()
    print(f"{index_list[i]} {modified_text}")
    

8 neighboring fields
16 but i will be in a minute
65 will you
100 do they
102 dont you
105 i dont know
112 why is it
132 why not
140 goodbye
184 the idea
